In [16]:
import pandas as pd
pd.options.mode.chained_assignment=None
import datetime as dt
import numpy as np
import mariadb
import os
from dotenv import load_dotenv
import sqlalchemy
import matplotlib.pyplot as plt
import itertools
from src.Tag import Tag
%load_ext autoreload
%autoreload 2

load_dotenv() 

mydb = mariadb.connect(
    host = os.getenv('HOST'),
    user = os.getenv('USER'),
    passwd = os.getenv('PASSWD'),
    database = os.getenv('DATABASE')
    )

def query_data_old(table,start_from,end_at):
    query = 'select * from {} WHERE LOCAL_TIME >= \'{}\' and LOCAL_TIME <= \'{}\';'.format(table,start_from,end_at)
    return pd.read_sql(query, con=mydb)

df = query_data_old('cleaned_data','2022-07-01','2022-07-02') 
df['AREA_NAME'] = df['AREA_NAME'].fillna('No area specified')
df = df.sort_values(by=['TAG_ID', 'EPOCHTIME'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Users\320159994\AppData\Local\miniforge3\envs\contacttracing\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [17]:
tags = [ 17420241, 17420328,17335070,17335272,17335306,17357876,17358063,17358071] #

In [18]:
tags_dict = {}
for tag in tags:
    print(tag)
    tags_dict[tag] = Tag(df, tag)
    
combinations = list(itertools.combinations(tags_dict, 2))

17420241
17420328
17335070
17335272
17335306
17357876
17358063
17358071


In [25]:
df_all = pd.DataFrame(columns=['TAG_ID_a', 'TAG_ID_b', 'start_time', 'end_time', 'overlap_minutes', 'MONITOR_ID', 'ZONE_NAME_a', 'ZONE_NAME_b',  'LOCAL_TIME_a', 'LOCAL_TIME_LAST_REPORT_IN_ZONE_a', 'LOCAL_TIME_b', 'LOCAL_TIME_LAST_REPORT_IN_ZONE_b',])
for tag_couple in combinations:
    x = tags_dict[tag_couple[0]].getIntersection(tags_dict[tag_couple[1]])
    df_all = pd.concat([df_all, x], ignore_index=True,sort=False)

There is no contact between the assets!
There is no contact between the assets!
There is no contact between the assets!
There is no contact between the assets!
There is no contact between the assets!
There is no contact between the assets!
There is no contact between the assets!
There is no contact between the assets!
There is no contact between the assets!
There is no contact between the assets!
There is no contact between the assets!
There is no contact between the assets!
There is no contact between the assets!
There is no contact between the assets!


In [26]:
df_all.sort_values('overlap_minutes',ascending=False)

,TAG_ID_a,TAG_ID_b,start_time,end_time,overlap_minutes,MONITOR_ID,ZONE_NAME_a,ZONE_NAME_b,LOCAL_TIME_a,LOCAL_TIME_LAST_REPORT_IN_ZONE_a,LOCAL_TIME_b,LOCAL_TIME_LAST_REPORT_IN_ZONE_b
9,17420241.0,17335070.0,2022-07-01 00:04:42,2022-07-01 14:13:47,849.083333,512765,Aufnahme,Aufnahme,2022-07-01 00:04:42,2022-07-01 14:47:38,2022-07-01 00:00:13,2022-07-01 14:13:47
44,17420328.0,17335070.0,2022-07-01 00:01:56,2022-07-01 10:17:48,615.866667,512765,Aufnahme,Aufnahme,2022-07-01 00:01:56,2022-07-01 10:17:48,2022-07-01 00:00:13,2022-07-01 14:13:47
7,17420241.0,17420328.0,2022-07-01 00:04:42,2022-07-01 10:17:48,613.100000,512765,Aufnahme,Aufnahme,2022-07-01 00:04:42,2022-07-01 14:47:38,2022-07-01 00:01:56,2022-07-01 10:17:48
48,17420328.0,17358071.0,2022-07-01 00:04:23,2022-07-01 09:50:44,586.350000,512765,Aufnahme,Aufnahme,2022-07-01 00:01:56,2022-07-01 10:17:48,2022-07-01 00:04:23,2022-07-01 09:50:44
54,17335070.0,17358071.0,2022-07-01 00:04:23,2022-07-01 09:50:44,586.350000,512765,Aufnahme,Aufnahme,2022-07-01 00:00:13,2022-07-01 14:13:47,2022-07-01 00:04:23,2022-07-01 09:50:44
...,...,...,...,...,...,...,...,...,...,...,...,...
25,17420328.0,17335070.0,2022-07-01 16:02:16,2022-07-01 16:02:16,0.000000,513929,Flur Chirurgie 2,Flur Chirurgie 2,2022-07-01 15:54:19,2022-07-01 16:24:32,2022-07-01 16:02:16,2022-07-01 16:02:16
26,17420328.0,17335070.0,2022-07-01 16:03:31,2022-07-01 16:03:31,0.000000,513929,Flur Chirurgie 2,Flur Chirurgie 2,2022-07-01 15:54:19,2022-07-01 16:24:32,2022-07-01 16:03:31,2022-07-01 16:03:31
27,17420328.0,17335070.0,2022-07-01 16:09:17,2022-07-01 16:09:17,0.000000,513929,Flur Chirurgie 2,Flur Chirurgie 2,2022-07-01 15:54:19,2022-07-01 16:24:32,2022-07-01 16:09:17,2022-07-01 16:09:17
60,17358063.0,17358071.0,2022-07-01 10:01:03,2022-07-01 10:01:03,0.000000,512765,Aufnahme,Aufnahme,2022-07-01 06:37:47,2022-07-01 11:25:14,2022-07-01 10:01:03,2022-07-01 10:01:03


In [ ]:
#add code here to check for overlap between two assets given just the number, maybe a UI???


In [ ]:
def compress(df):
    """
    Removes subsequent reports of identical locations; 
    Calculates TIME_IN_ZONE calculated as timestamp first report - timestap last report
    
    Parameters: 
    df (pandas dataframe)
    
    Output:
    temp (pandas dataframe): compressed dataframe
                   
    """
    temp = df
    if not df.empty:
        temp = temp.sort_values(by=['TAG_ID', 'EPOCHTIME'])  
        temp = temp[temp.MONITOR_ID != 0]
        #keep monitor id if it is not equal to the previous or next one and keep tag id if it is equal to previous or next one
        temp['keep'] = temp['keep'] = (((temp.TAG_ID == temp.TAG_ID.shift(1))|(temp.TAG_ID == temp.TAG_ID.shift(-1))) & ((temp.MONITOR_ID != temp["MONITOR_ID"].shift(1)) | (temp.MONITOR_ID != temp["MONITOR_ID"].shift(-1))))
        temp['keep'].iloc[0] = True
        temp = temp[temp.keep == True]
        temp['TIME_LAST_REPORT_IN_ZONE'] = np.where((temp.MONITOR_ID == temp.MONITOR_ID.shift(-1)) & (temp.TAG_ID == temp.TAG_ID.shift(-1)) ,temp.EPOCHTIME.shift(-1),temp.EPOCHTIME)
        temp['LOCAL_TIME_LAST_REPORT_IN_ZONE'] = np.where((temp.MONITOR_ID == temp.MONITOR_ID.shift(-1)) & (temp.TAG_ID == temp.TAG_ID.shift(-1)) ,temp.LOCAL_TIME.shift(-1),temp.LOCAL_TIME)
        #remove duplicate monitor ids
        temp['remove'] = ((temp.TAG_ID == temp.TAG_ID.shift(1)) & (temp.MONITOR_ID == temp["MONITOR_ID"].shift(1)))
        temp = temp[temp.remove == False]
        #calculate time in zone
        temp['TIME_IN_ZONE'] = temp.TIME_LAST_REPORT_IN_ZONE - temp.EPOCHTIME
    return(temp)

x = df.loc[df['TAG_ID']==1984]
compress(x)

In [ ]:
# tag_17358071 = Tag(df, 17358071)
# tag_17358113 = Tag(df, 17358113)

tag_17420241 = Tag(df, 17420241)
tag_17420328 = Tag(df, 17420328)

In [ ]:
x = tag_17420241.getIntersection(tag_17420328)

In [ ]:
x['start_time'] = x.apply(lambda row: max(row['LOCAL_TIME_a'], row['LOCAL_TIME_b']), axis=1)
x['end_time'] = x.apply(lambda row: min(row['LOCAL_TIME_LAST_REPORT_IN_ZONE_a'], row['LOCAL_TIME_LAST_REPORT_IN_ZONE_b']), axis=1)
x['overlap'] = (x['end_time'] - x['start_time'])
x['overlap'] = x['overlap'].dt.total_seconds()
x = x.sort_values(by='overlap')
x[['TAG_ID_a', 'MONITOR_ID', 'LOCAL_TIME_a', 'LOCAL_TIME_LAST_REPORT_IN_ZONE_a', 'LOCAL_TIME_b', 'LOCAL_TIME_LAST_REPORT_IN_ZONE_b', 'ZONE_NAME_a', 'ZONE_NAME_b', 'start_time', 'end_time', 'overlap']].dropna()

In [ ]:
tags = [tag_17420241, tag_17420328]

In [ ]:
combos = list(itertools.combinations(tags, 2))

In [ ]:

# def fu(row):
#     start_time = row['LOCAL_TIME_a'] if row['LOCAL_TIME_a'] > row['LOCAL_TIME_b'] else row['LOCAL_TIME_b']
#     end_time = row['LOCAL_TIME_LAST_REPORT_IN_ZONE_a'] if row['LOCAL_TIME_LAST_REPORT_IN_ZONE_a']< row['LOCAL_TIME_LAST_REPORT_IN_ZONE_b'] else row['LOCAL_TIME_LAST_REPORT_IN_ZONE_b']
#     return (end_time-start_time).total_seconds()/60

# x['overlap']=x.apply(lambda row : fu(row))
# x

In [ ]:
x = tag_17420241.data.copy()
x = x.loc[x['MOTION']!=0]
x['LOCAL_TIME'] = pd.to_datetime(x['LOCAL_TIME'])

x['delta'] = (x['LOCAL_TIME']-x['LOCAL_TIME'].shift())

def fu(row):
    return row.total_seconds()

x['delta']=x['delta'].apply(lambda row : fu(row))

In [ ]:

ax1 = plt.subplot(211)
ax1.scatter(x['LOCAL_TIME'], x['MOTION'],color='b',marker='x')
ax1.set_ylim(0,1)

ax2 = plt.subplot(212)
ax2.scatter(x['LOCAL_TIME'], x['delta'],color='r',marker='o')
ax2.set_ylim([x['delta'].min(), x['delta'].max()])
plt.show()

In [ ]:
def compress(df):
    """
    Removes subsequent reports of identical locations; 
    Calculates TIME_IN_ZONE calculated as timestamp first report - timestap last report
    
    Parameters: 
    df (pandas dataframe)
    
    Output:
    temp (pandas dataframe): compressed dataframe
                   
    """
    temp = df
    if not df.empty:
        temp = temp.sort_values(by=['TAG_ID', 'EPOCHTIME'])  
        temp = temp[temp.MONITOR_ID != 0]
        #keep monitor id if it is not equal to the previous or next one and keep tag id if it is equal to previous or next one
        temp['keep'] = temp['keep'] = (((temp.TAG_ID == temp.TAG_ID.shift(1))|(temp.TAG_ID == temp.TAG_ID.shift(-1))) & ((temp.MONITOR_ID != temp["MONITOR_ID"].shift(1)) | (temp.MONITOR_ID != temp["MONITOR_ID"].shift(-1))))
        temp['keep'].iloc[0] = True
        temp = temp[temp.keep == True]
        temp['TIME_LAST_REPORT_IN_ZONE'] = np.where((temp.MONITOR_ID == temp.MONITOR_ID.shift(-1)) & (temp.TAG_ID == temp.TAG_ID.shift(-1)) ,temp.EPOCHTIME.shift(-1),temp.EPOCHTIME)
        temp['LOCAL_TIME_LAST_REPORT_IN_ZONE'] = np.where((temp.MONITOR_ID == temp.MONITOR_ID.shift(-1)) & (temp.TAG_ID == temp.TAG_ID.shift(-1)) ,temp.LOCAL_TIME.shift(-1),temp.LOCAL_TIME)
        #remove duplicate monitor ids
        temp['remove'] = ((temp.TAG_ID == temp.TAG_ID.shift(1)) & (temp.MONITOR_ID == temp["MONITOR_ID"].shift(1)))
        temp = temp[temp.remove == False]
        #calculate time in zone
        temp['TIME_IN_ZONE'] = temp.TIME_LAST_REPORT_IN_ZONE - temp.EPOCHTIME
    else:
        temp['TIME_IN_ZONE'] = np.nan
        temp['TIME_LAST_REPORT_IN_ZONE'] = np.nan
        temp['LOCAL_TIME_LAST_REPORT_IN_ZONE'] = np.nan
    return(temp)


tag_17358071 = Tag(df, 17358071)
tag_17358113 = Tag(df, 17358113)


tag_1_compressed = compress(tag_17358071.data) 
df_A = tag_1_compressed.rename(columns={'LOCAL_TIME': 'start_date', 'LOCAL_TIME_LAST_REPORT_IN_ZONE':'end_date'})
df_A.head()

tag_2_compressed = compress(tag_17358113.data) 
df_B = tag_2_compressed.rename(columns={'LOCAL_TIME': 'start_date', 'LOCAL_TIME_LAST_REPORT_IN_ZONE':'end_date'})
df_B

for df in [df_A, df_B]:
    for k in ['start_date', 'end_date']:
        df[k] = pd.to_datetime(df[k])

# next, merge on eventname, regardless of interval overlapping
zo = df_A.merge(df_B, on='MONITOR_ID', suffixes=['_a', '_b'], how='outer')

# finally, transform that result to dissociate non-overlapping groups
def proc(g):
    assert len(g) == 1
    #converting to series
    r = g.squeeze()
    try:
        a = pd.Interval(r['start_date_a'], r['end_date_a'])
        b = pd.Interval(r['start_date_b'], r['end_date_b'])
        if a.overlaps(b):
            return g
        # emit two rows instead
        cols = ['start_date_a', 'end_date_a', 'MONITOR_ID', 'start_date_b', 'end_date_b']
        sa, ea, ev, sb, eb = g[cols]
        return pd.DataFrame([
            [pd.NaT, pd.NaT, ev, sb, eb],
            [sa, ea, ev, pd.NaT, pd.NaT],
        ], columns=cols)
    except ValueError:
        return g

# out = zo.groupby(level=0).apply(proc).reset_index(drop=True)
# pd.set_option('display.max_columns', 500)
# out[['MONITOR_ID', 'start_date_a', 'end_date_a', 'start_date_b', 'end_date_b', 'ZONE_NAME_a', 'ZONE_NAME_b']].dropna()

In [ ]:
compressed_df = compress(df)
compressed_df.to_csv('compressed_df_01_07_22.csv')